In [1]:
from collections import OrderedDict
import pandas as pd
from config import DSIN_SESS_COUNT, DSIN_SESS_MAX_LEN, FRAC
from  model import DSIN
from tensorflow.python.keras.layers import Input

g:\python\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
g:\python\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
g:\python\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
g:\python\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; 

In [10]:
def create_singlefeat_inputdict(feature_dim_dict, prefix=''):
    sparse_input = OrderedDict()
    for feat in feature_dim_dict["sparse"]:
        sparse_input[feat.name] = Input(
            shape=(1,), name=prefix+feat.name, dtype="float32")

    dense_input = OrderedDict()

    for  feat in feature_dim_dict["dense"]:
        dense_input[feat.name] = Input(
            shape=(1,), name=prefix+feat.name,dtype="float32")

    return sparse_input, dense_input
def get_input(feature_dim_dict, seq_feature_list, sess_max_count, seq_max_len):
    sparse_input, dense_input = create_singlefeat_inputdict(feature_dim_dict)
    user_behavior_input = {}
    for idx in range(sess_max_count):
        sess_input = OrderedDict()
        for i, feat in enumerate(seq_feature_list):
            sess_input[feat] = Input(
                shape=(seq_max_len,), name='seq_' + str(idx) + str(i) + '-' + feat)

        user_behavior_input["sess_" + str(idx)] = sess_input

    user_behavior_length = {"sess_" + str(idx): Input(shape=(1,), name='seq_length' + str(idx)) for idx in
                            range(sess_max_count)}
    user_sess_length = Input(shape=(1,), name='sess_length')

    return sparse_input, dense_input, user_behavior_input, user_behavior_length, user_sess_length

In [6]:
SESS_COUNT = DSIN_SESS_COUNT
SESS_MAX_LEN = DSIN_SESS_MAX_LEN
fd_path = './model_input/dsin_fd_' + str(FRAC) + '_' + str(SESS_COUNT) + '.pkl'
input_path = './model_input/dsin_input_' + str(FRAC) + '_' + str(SESS_COUNT) + '.pkl'
sample_sub_path = './model_input/raw_sample_'+ str(FRAC) + _ + '.pkl'
label_path = './model_input/dsin_label_' + str(FRAC) + '_' + str(SESS_COUNT) + '.pkl'
fd = pd.read_pickle(fd_path)
model_input = pd.read_pickle(input_path)
label = pd.read_pickle(label_path)

sample_sub = pd.read_pickle(sample_sub_path)
sample_sub['idx'] = list(range(sample_sub.shape[0]))
train_idx = sample_sub.loc[sample_sub.time_stamp <1494633600, 'idx'].values[:17800]
test_idx = sample_sub.loc[sample_sub.time_stamp >=1494633600, 'idx'].values[:2100]

train_input = [i[train_idx] for i in model_input]
test_input = [i[test_idx] for i in model_input]

In [12]:
sess_count = SESS_COUNT
sess_len_max = SESS_MAX_LEN
BATCH_SIZE = 1024

sess_feature = ['cate_id', 'brand']

In [13]:
sparse_input, dense_input, user_behavior_input_dict, _, user_sess_length = get_input(
        fd, sess_feature, sess_count, sess_len_max)

In [15]:
user_sess_length

<tf.Tensor 'sess_length:0' shape=(?, 1) dtype=float32>

In [1]:
import numpy as np
import tensorflow as tf

g:\python\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
g:\python\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
g:\python\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
g:\python\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; 

In [13]:
t = np.ones(shape=[2,3,4])

In [18]:
b = np.ones(shape = [1,1,4])
c = np.ones(shape = [1,3,4])
d = np.ones(shape = [4,4])

In [19]:
e = d+d

In [27]:
f = tf.tensordot(t,e,axes=(-1, 0))

In [30]:
f

<tf.Tensor 'Tensordot_1:0' shape=(2, 3, 4) dtype=float64>

In [31]:
tf.split(f,4, axis=2)

[<tf.Tensor 'split_1:0' shape=(2, 3, 1) dtype=float64>,
 <tf.Tensor 'split_1:1' shape=(2, 3, 1) dtype=float64>,
 <tf.Tensor 'split_1:2' shape=(2, 3, 1) dtype=float64>,
 <tf.Tensor 'split_1:3' shape=(2, 3, 1) dtype=float64>]

In [32]:
g = tf.concat(tf.split(f,4, axis=2),axis=0)

In [34]:
e = np.ones(shape=[2,3,4])

In [35]:
tf.matmul(e, e, transpose_b=True)

<tf.Tensor 'MatMul:0' shape=(2, 3, 3) dtype=float64>

In [39]:
e.get_shape().as_list()[-1]

4

In [37]:
e= tf.convert_to_tensor(e)

In [56]:
query_masks = np.ones([3,4])*2

In [57]:
query_masks

array([[2., 2., 2., 2.],
       [2., 2., 2., 2.],
       [2., 2., 2., 2.]])

In [43]:
tf.sequence_mask(
              e, 3, dtype=tf.float32)

<tf.Tensor 'SequenceMask_1/Cast_1:0' shape=(2, 3, 4, 3) dtype=float32>

In [54]:
mask = tf.sequence_mask(e, 5)

In [55]:
with tf.Session() as sess:
    mask = sess.run(mask)
    print(mask,mask.shape,e)

[[[[ True False False False False]
   [ True False False False False]
   [ True False False False False]
   [ True False False False False]]

  [[ True False False False False]
   [ True False False False False]
   [ True False False False False]
   [ True False False False False]]

  [[ True False False False False]
   [ True False False False False]
   [ True False False False False]
   [ True False False False False]]]


 [[[ True False False False False]
   [ True False False False False]
   [ True False False False False]
   [ True False False False False]]

  [[ True False False False False]
   [ True False False False False]
   [ True False False False False]
   [ True False False False False]]

  [[ True False False False False]
   [ True False False False False]
   [ True False False False False]
   [ True False False False False]]]] (2, 3, 4, 5) Tensor("Const_3:0", shape=(2, 3, 4), dtype=float64)


In [77]:
b = np.ones([2,5,5])

In [78]:
b

array([[[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]],

       [[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]]])

In [81]:
a = tf.matrix_set_diag(b, tf.ones_like(b)[:, :, 0] * 3)
c = tf.reduce_max(a, axis=-1, keep_dims=True)
w = tf.nn.softmax(c)

In [83]:
with tf.Session() as sess:
    q = sess.run(w)
    print(q,q.shape)

[[[1.]
  [1.]
  [1.]
  [1.]
  [1.]]

 [[1.]
  [1.]
  [1.]
  [1.]
  [1.]]] (2, 5, 1)


In [6]:
a = tf.constant([[1, 2], [3, 4], [5, 6]], dtype=tf.float32)
b = tf.constant([[1, 2,2], [3, 4,1], [5, 6,4]], dtype=tf.float32)
e = tf.constant([[1, 2,2,3], [3, 4,1,1], [5, 6,4,2]], dtype=tf.float32)
c = tf.concat([a,b,e],axis=-1)
#a1 = tf.tile(a, [1, 5])
with tf.Session() as sess:
    print(sess.run(a))
    print(sess.run(tf.shape(a)))
    print(sess.run(tf.shape(b)))
    print(sess.run(c))
    
    print(sess.run(tf.shape(c)))

[[1. 2.]
 [3. 4.]
 [5. 6.]]
[3 2]
[3 3]
[[1. 2. 1. 2. 2. 1. 2. 2. 3.]
 [3. 4. 3. 4. 1. 3. 4. 1. 1.]
 [5. 6. 5. 6. 4. 5. 6. 4. 2.]]
[3 9]
